In [15]:
# libraries

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import ast
from sklearn.model_selection import train_test_split


In [16]:
#df1 = pd.read_csv("data/large_board_dataset.csv")
df2 = pd.read_csv("data/large_board_dataset2.csv")
df3 = pd.read_csv("data/large_board_dataset3.csv")
#df4 = pd.read_csv("data/large_board_dataset2-mikala.csv")
#df4 = pd.read_csv("data/player_one_moves.csv")
#df5 = pd.read_csv("data/player_two_moves.csv")


df = pd.concat([df2, df3], axis =0)


In [17]:
df.head()
df["x"] = df["x"].apply(lambda x: np.array(ast.literal_eval(x)).reshape(2, 6, 7))


In [18]:
def preprocess_board(board):
    if board["whose_turn"] != "red":
        # Flip the layers to make each board look like it's from the perspective of "red", aka plus
        board["x"] = board["x"][::-1, :, :]
    return board

df = df.apply(preprocess_board, axis=1)

In [19]:
def flip_board(board, col):
    new_board = np.flip(board, axis = 2)
    # for now this needs to be done by index 2 because the input is still 2x6x7, but this can easily be updated
    new_column = 6 - col
    return new_board, new_column

flipped_x = []
flipped_y = []

for index, row in df.iterrows():
    board = row["x"]
    col = row["y"]
    new_board, new_column = flip_board(board, col)

    flipped_x.append(new_board)
    flipped_y.append(new_column)

new_df = pd.DataFrame({"x": flipped_x, "y": flipped_y})

df = pd.concat([df, new_df], ignore_index = True)

In [20]:
df.to_csv("data/2_and_3_combined.csv")

In [21]:
boards = df["x"]
columns = df["y"]

np.savez("data/final_data_jan_27.npz", X = boards, y = columns)

In [22]:
# the below demonstrate that the flip_board() function works

df["x"][0]

array([[[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 1.]],

       [[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0., 1., 0.]]])

In [23]:
flip_board(df["x"][0], 0)

(array([[[0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0.],
         [1., 0., 1., 0., 0., 0., 0.]],
 
        [[0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 1., 0., 1., 0.]]]),
 6)

In [24]:
x = df["x"]
y = df["y"]

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 22)

X_train = np.stack(X_train)
X_test = np.stack(X_test)
y_train = np.array(y_train)  
y_test = np.array(y_test)

X_train = X_train.transpose(0, 2, 3, 1)  # Convert (num_samples, 2, 6, 7) to (num_samples, 6, 7, 2)
X_test = X_test.transpose(0, 2, 3, 1)

In [25]:
cnn = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(filters=64,kernel_size=(4,4),activation=tf.nn.relu,input_shape=(6, 7, 2)),
        # had to adjust data_format because it was treating the images as 2x6 with 7 channels
        tf.keras.layers.MaxPooling2D(pool_size = (2,1),strides=(1, 1), padding = "same"),
        tf.keras.layers.Conv2D(64, (2, 2), activation=tf.nn.relu),
        tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides = (1, 1), padding = "same"),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64,activation=tf.nn.relu,kernel_regularizer = tf.keras.regularizers.l2(0.005)),
        tf.keras.layers.Dropout(0.1), 
        tf.keras.layers.Dense(64,activation=tf.nn.relu,kernel_regularizer = tf.keras.regularizers.l2(0.001)),
        tf.keras.layers.Dropout(0.4), 
        tf.keras.layers.Dense(7,activation=tf.nn.softmax)
        ])

/Users/fmunting/opt/anaconda3/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [26]:
cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

cnn.summary()

cnn.fit(X_train,y_train,epochs=200,validation_split=0.2,batch_size=125)

pred_probs = cnn.predict(X_test)
pred = np.argmax(pred_probs, axis=1)
print(np.mean(pred==y_test))

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 3, 4, 64)       │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 3, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 2, 3, 64)       │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 2, 3, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 384)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │        24,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 7)              │           455 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 47,815 (186.78 KB)

 Trainable params: 47,815 (186.78 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/200
1336/1336 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.2796 - loss: 1.9019 - val_accuracy: 0.4535 - val_loss: 1.4986
Epoch 2/200
1336/1336 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.4560 - loss: 1.5094 - val_accuracy: 0.4933 - val_loss: 1.3908
Epoch 3/200
1336/1336 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - accuracy: 0.4853 - loss: 1.4204 - val_accuracy: 0.5028 - val_loss: 1.3542
Epoch 4/200
1336/1336 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.4994 - loss: 1.3819 - val_accuracy: 0.5148 - val_loss: 1.3143
Epoch 5/200
1336/1336 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - accuracy: 0.5113 - loss: 1.3477 - val_accuracy: 0.5236 - val_loss: 1.2937
Epoch 6/200
1336/1336 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.5174 - loss: 1.3245 - val_accuracy: 0.5253 - val_loss: 1.2761
Epoch 7/200
1336/1336 ━━━━━━━━━━━━━━━━━━━━ 13s 10ms/step - accuracy: 0.5249 - loss: 1.3091 - val_accuracy: 0.5328 - val_loss: 1.2578
Epoch 8/200
1336/1336 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.5276 

KeyboardInterrupt: 

In [ ]:
df.to_csv("data/2_and_3_combined.csv")